In [1]:
#Imports
import pandas as pd
import datetime
import os

In [9]:
# Get current working directory
cwd = os.getcwd()
print(f'File Path: {cwd}')

# Create path to mario_data (go up one level, then into mario_data)
import os
data_path = os.path.join(os.path.dirname(cwd), 'mario_data')
print(f'Data Location: {data_path}')

# Verify that folder actually exists
print(os.path.exists(data_path))


File Path: /Users/michaelsobnosky/Desktop/Medallion Architecture/Bronze
Data Location: /Users/michaelsobnosky/Desktop/Medallion Architecture/mario_data
True


In [10]:
#Get all folders and files as a list
folder_files = os.listdir(data_path)
for f in folder_files:
    print(f)

mario_data_20250601.jsonl
mario_data_20250401.jsonl
mario_data_20250501.jsonl
mario_data_20250201.csv
mario_data_20250301.csv
mario_data_20250101.csv


In [12]:
# Create a data frame with pandas
df_peek = pd.read_csv(data_path + '/mario_data_20250201.csv')

# Read the first 5 rows
df_peek.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game
0,NaN,Sarasaland Stars,Toad Town,Comet Bike,Goomba,B,C,C,18,Blocker,3.0,3.0000,No,No,Banana Peel,479,68,3.00,Super Mario World
1,Bowser Jr.,Mushroom Royalty,Koopa Kingdom,Circuit Special,Shy Guy,C,D,S,11,Driver,174.0,1.0000,No,Yes,1-Up Mushroom,439,67,7.00,Mario Strikers
2,Toadette,Red Caps,Mushroom Kingdom,Comet Bike,Goomba,A,B,S,13,Item Specialist,-25.0,5.9460,No,No,Super Mushroom,161,66,6.09,Mario Tennis Aces
3,Daisy,Red Caps,Toad Town,Biddybuggy,Koopa Troopa,A,B,A,9,Item Specialist,-93.0,2.0000,Yes,Yes,"Fire Flower, Red Shell, 1-Up Mushroom",269,36,4.00,Mario Kart 8 Deluxe
4,Rosalina,Sarasaland Stars,Star World,Standard Kart,Shy Guy,S,D,C,19,Blocker,152.0,2.1941,No,No,"Green Shell, Super Mushroom",500,61,10.00,Super Mario Odyssey


In [15]:
# Combine all csvs
# Create empty variable to add all data frames into a list
df_list = []

for file in folder_files:
    print(f"Reading {file}")
    file_path = os.path.join(data_path, file)  # cross-platform path

     # choose the right reader based on file extension
    if file.lower().endswith(".csv"):
        temp_df = pd.read_csv(file_path)
    elif file.lower().endswith(".jsonl"):
        temp_df = pd.read_json(file_path)
    else:
        print(f"Skipping unsupported file: {file}")
        continue
    
    temp_df["fileName"] = file # Add file name
    temp_df["loadDatetimeStamp"] = datetime.datetime.now() # Add datetime stamp

    df_list.append(temp_df) 

# combine all files into a single DataFrame
df = pd.concat(df_list, ignore_index=True)
print("Completed DataFrame concatenation:", df.shape)

# Peek at data
df.head()
df.shape
df["fileName"].value_counts()

Reading mario_data_20250601.jsonl
Reading mario_data_20250401.jsonl
Reading mario_data_20250501.jsonl
Reading mario_data_20250201.csv
Reading mario_data_20250301.csv
Reading mario_data_20250101.csv
Completed DataFrame concatenation: (48000, 21)


fileName
mario_data_20250601.jsonl    8000
mario_data_20250401.jsonl    8000
mario_data_20250501.jsonl    8000
mario_data_20250201.csv      8000
mario_data_20250301.csv      8000
mario_data_20250101.csv      8000
Name: count, dtype: int64

In [18]:
df.to_parquet('bronze_mario.parquet', index=False) # Save to bronze layer